# Mistral-7B-Instruct 모델_End-to-End 추가

## "학습” 안 하는 이유
1. 데이터 로딩/전처리:
의료 QA 데이터 등 → 텍스트 조각(Document)으로 분할 및 필터링
2. 임베딩:
Ko-SBERT 등 “사전학습된 임베딩 모델”로 텍스트를 벡터로 변환
(여기서 학습이 아니라 “이미 학습된 모델로 변환”만 하는 것)
3. 벡터DB 생성:
FAISS에 임베딩 벡터 저장
4. 질의응답:
사용자가 질문하면 →
    - (1) 벡터DB에서 유사한 문서 검색
    - (2) Mistral-7B-Instruct 등 대형 언어모델(역시 “사전학습된”)이
검색된 문서 조각들을 참고해서 답변 생성
## 실제 “학습”이란?
- 파인튜닝/미세조정 등은
모델을 직접 추가로 훈련시키는 것 (모델 파라미터가 변함)
- 예시: model.fit(), trainer.train(), “epochs”, “loss” 등이 코드에 나옴

지금 코드의 RAG 구조는:
- 임베딩 모델(KoSBERT): 이미 학습된 걸 “불러와서” 임베딩만 실행
- LLM(Mistral-7B-Instruct 등): 이미 학습된 걸 “불러와서” 답변 생성만 하는 것
- 파인튜닝(학습) 없음

## 요약
“임베딩/벡터DB/질의응답 파이프라인”이지
모델 파인튜닝(학습)은 안 하는 구조

# “RAG 없는 LLM 챗봇”
1. 학습(파인튜닝)이 필요한 경우가 많음
2. LLM(예: Mistral, Llama, GPT 등)만 놓고 사용하면
- → 사전학습 범위 내의 지식만 답변
- → 회사/기관/도메인 고유 정보, 최신 정보 반영 불가
- → 답변 품질, 일관성 낮음

3. **우리 데이터/지식으로 LLM을 ‘맞춤화’**하고 싶으면
→ 파인튜닝(학습)이 필요

# “RAG 있는 LLM 챗봇”
1. 학습(파인튜닝) 없이도 실제 업무·실전 서비스 가능
- 내·외부 문서, 자료를 벡터DB로 embedding해서
- “검색-답변(Retrieval + Generation)” 방식으로 동작
- LLM은 문서에서 답을 찾아서 생성
- 우리 데이터만 잘 embedding하면 바로 적용
(LLM은 사전학습된 상태 그대로 사용)

# 정리
1. RAG 없음 → 파인튜닝 거의 필수
(실무 적용/특화된 답변 원할 때)

2. RAG 있음 → 파인튜닝 필요 없음
(거의 모든 도메인 챗봇, 사내 QA 가능)

# RAG는 “검색+생성” 구조라 우리 데이터를 LLM에 직접 넣어주지 않아도, 최신/도메인 정보로 답변 가능

====================================================================================================================================================================================================

In [ ]:
# QA 데이터셋 구조 확인
import os, json

label_dirs = [
    "dataset/medical_knowledge_QA/Training/02.라벨링데이터",
    "dataset/medical_knowledge_QA/Validation/02.라벨링데이터",
]

e2e_train = []
for label_dir in label_dirs:
    for filename in os.listdir(label_dir):
        if filename.endswith(".json"):
            file_path = os.path.join(label_dir, filename)
            with open(file_path, encoding="utf-8-sig") as f:
                data = json.load(f)
                # 데이터 구조: data["question"], data["answer"], data["context"] (context 없으면 "" 또는 없음)
                question = data.get("question", "")
                answer = data.get("answer", "")
                context = data.get("context", "")
                if question and answer:
                    e2e_train.append(
                        {
                            "question": question,
                            "context": context,
                            "answer": answer,
                        }
                    )

print("총 QA 샘플 수:", len(e2e_train))
print("첫 번째 샘플:", e2e_train[0])

## 임포트 및 환경 설정

In [1]:
import os, json, re
import pandas as pd
from tqdm import tqdm
from typing import List
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from PyPDF2 import PdfReader

In [2]:
# GPU 확인용 코드
# CPU 사용시 느림 현상으로 멈춤 => GPU 사용
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
1
NVIDIA GeForce RTX 4070


## 2. 데이터 불러오기

In [3]:
# 1) 네이버 JSON
with open("dataset/naver_docs.json", encoding="utf-8") as f:
    naver = json.load(f)
# 2) AMC JSON
with open("dataset/amc_docs.json", encoding="utf-8") as f:
    amc = json.load(f)
# 3) medical_knowledge_QA
label_dirs = [
    "dataset/medical_knowledge_QA/Training/02.라벨링데이터",
    "dataset/medical_knowledge_QA/Validation/02.라벨링데이터",
]
all_qa_data = []
for label_dir in label_dirs:
    for filename in os.listdir(label_dir):
        if filename.endswith(".json"):
            file_path = os.path.join(label_dir, filename)
            with open(file_path, encoding="utf-8-sig") as f:
                try:
                    data = json.load(f)
                    all_qa_data.append(data)
                except Exception as e:
                    print(f"[오류] {filename}: {e}")

# 4) medical_legal_corpus 폴더
text_dir = "dataset/medical_legal_corpus/Training/01.원천데이터"
all_texts = []
for filename in os.listdir(text_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(text_dir, filename)
        with open(file_path, encoding="utf-8-sig") as f:
            text = f.read()
            all_texts.append({"filename": filename, "text": text})

# 5) 라벨링 QA (End-to-End 파인튜닝 데이터 준비)
label_path = (
    "dataset/medical_legal_corpus/Training/02.라벨링데이터/Training_medical.json"
)
with open(label_path, encoding="utf-8-sig") as f:
    legal_qa_data = json.load(f)
qa_items = legal_qa_data.get("data", [])
e2e_train = []
for qa in qa_items:
    question = qa.get("question", "")
    context = qa.get("context", "")
    answer = qa.get("answer", "")
    if question and context and answer:
        e2e_train.append({"question": question, "context": context, "answer": answer})

# 6) PDF → 텍스트 추출 및 분할 (필요시)
pdf_path = "dataset/antibiotic_guideline_for_longtermcare.pdf"
reader = PdfReader(pdf_path)
all_text = ""
for page in reader.pages:
    all_text += page.extract_text() + "\n"

## 3. 텍스트 병합

In [4]:
texts = []
# 네이버, 아산병원
texts += [doc["text"] for doc in naver]
texts += [doc["text"] for doc in amc]
# medical_knowledge_QA
for item in all_qa_data:
    if isinstance(item, list):
        for qa in item:
            q = qa.get("question", "")
            a = qa.get("answer", "")
            if q or a:
                texts.append(q + "\n" + a)
# medical_legal_corpus
texts += [doc["text"] for doc in all_texts if isinstance(doc, dict) and "text" in doc]
# 라벨링 QA
texts += [qa.get("text", "") for qa in qa_items if "text" in qa]
# PDF
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = splitter.split_text(all_text)
texts += chunks

## 4. 텍스트 분할 및 전처리

In [5]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800, chunk_overlap=100, separators=["\n\n", "\n", " ", ""]
)
documents = splitter.create_documents(texts)

# 전처리(예: 너무 짧거나, 유전자서열/스팸/광고 제거, 키워드 필터링)
MEDICAL_KEYWORDS = [
    "병원",
    "의과",
    "내과",
    "이비인후과",
    "피부과",
    "정형외과",
    "신경과",
    "정신과",
    "산부인과",
    "응급실",
    "진료",
    "의사",
    "간호사",
    "환자",
    "보호자",
    "수술",
    "진단",
    "시술",
    "검사",
    "CT",
    "MRI",
    "X-ray",
    "혈액",
    "소변",
    "조영제",
    "항생제",
    "주사",
    "투약",
    "약물",
    "처방",
    "상담",
    "증상",
    "발열",
    "기침",
    "두통",
    "감기",
    "인후통",
    "복통",
    "설사",
    "요로감염",
    "폐렴",
    "당뇨",
    "고혈압",
    "암",
    "코로나",
    "독감",
    "백신",
    "면역",
    "질환",
    "질병",
    "감염",
    "감염병",
    "폐렴",
    "요로감염",
    "피부염",
    "욕창",
    "MRSA",
    "바이러스",
]
clean_documents = []
for doc in documents:
    text = doc.page_content.strip()
    if len(text) < 10:
        continue
    if re.fullmatch(r"[ACGT\n]+", text):
        continue
    if any(word in text for word in MEDICAL_KEYWORDS):
        clean_documents.append(doc)
documents = clean_documents
print(f"전처리 후 문서 수: {len(documents)}")

전처리 후 문서 수: 308545


## 5. KoSBERT 임베딩 및 FAISS 벡터DB

In [6]:
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")
db = FAISS.from_documents(documents, embedding=embedding_model)
db.save_local("faiss_medical_knowledge")
print("FAISS 벡터 DB 저장 완료!")

C:\Users\user\AppData\Local\Temp\ipykernel_23028\3877256484.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")
c:\Users\user\Desktop\medical_chatbot\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


FAISS 벡터 DB 저장 완료!


In [7]:
print(documents[:3])

[Document(metadata={}, page_content='가랑이통증 [Perineal pain]은(는) 원인에 따라 증상에 다소 차이가 있을 수 있다. *요로감염:  배뇨통  및 빈뇨,  절박뇨 , 골반부위의 둔통 등 *항문주위농양: 발적과 종창, 압통 및 욱신거리는 통증을 동반한 덩이(감염에 의한 경우 열 및 오한을 동반할 수 있음) *음부신경포착증후군: 앉은자세일때 심해지는 통증 *골반바닥근육의 이상: 배뇨, 배변시의 장애 동반 등 증상이 나타나는 질환이며, 관련 진료과는 산부인과입니다.'), Document(metadata={}, page_content='가성근시 [pseudomyopia]은(는) 원거리 시력저하가 나타나고 과도한 모양체 근육의 수축으로 인해서 눈이 피로하고 안구통증, 두통, 어지러움 등이 나타날 수 있다. 증상이 나타나는 질환이며, 관련 진료과는 안과, 정신건강의학과입니다.'), Document(metadata={}, page_content='가스 괴저병 [gas gangrene]은(는) 가스 괴저병은 빠른 속도로 진행하며 생명을 위협할 수준까지 악화되기도 한다. 상처를 통해서 균이 침투한 후, 증상이 나타나기까지 걸리는 잠복기는 대개 1~4일 정도이다. 처음에는 상처 부위에 매우 심한 통증이 발생하고, 수분에서 수시간 사이에 빠르게 진행하여 감염 부위가 붓고 창백하게 변하며 누를 때 통증(동통)이 발생한다. 연부조직 부위에 차오르는 가스는 방사선 사진을 통해 눈으로 확인할 수 있고, 또는 손으로 눌러보면 가스가 발생하는 것을 알 수 있다. 가스 괴저병이 발생한 부위의 피부는 초기의 창백한 색조에서 점점 색깔이 변하여 구릿빛으로 진해지며, 때때로 출혈이 동반된 물집이 잡히기도 한다.  식은땀 이 나고 심장 박동수가 빨라지며 심하게 안절부절 못하는 등의 전신적인 증상을 동반하기도 한다. 조기에 진단하여 치료를 하지 않으면 병이 발생한 팔이나 다리를 절제해야 하며, 이 시기를 놓치면 사망에 이를 수 있다. 증상이 나타나는 질환이며,

## 6. LLM 로딩 및 QA 체인

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM


HF_TOKEN = "your_token_here"  # HuggingFace 토큰


model_name = "mistralai/Mistral-7B-Instruct-v0.2"


tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)


model = AutoModelForCausalLM.from_pretrained(
    model_name, token=HF_TOKEN, device_map="auto", torch_dtype="auto"
)


llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15,
)


llm = HuggingFacePipeline(pipeline=llm_pipeline)


db = FAISS.load_local(
    "faiss_medical_knowledge", embedding_model, allow_dangerous_deserialization=True
)


retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})


qa_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0
C:\Users\user\AppData\Local\Temp\ipykernel_23028\3400874234.py:21: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


## 7. End-to-End 파인튜닝 데이터셋 저장

In [10]:
# (추가로 train/val 분리, ColBERT 등 학습코드로 바로 넘길 수 있음)
from sklearn.model_selection import train_test_split

train, val = train_test_split(qa_items, test_size=0.1, random_state=42)

with open("medical_knowledge_QA_split.json", "w", encoding="utf-8-sig") as f:
    json.dump({"train": train, "validation": val}, f, ensure_ascii=False, indent=2)

print(f"train: {len(train)}, val: {len(val)}")

train: 33756, val: 3751


## 8. 실시간 QA 인터페이스 예시(ipywidgets 등)

8-1 답변

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

question_box = widgets.Text(
    value="",
    placeholder="여기에 질문을 입력하세요",
    description="질문:",
    disabled=False,
)
output_box = widgets.Output()
submit_btn = widgets.Button(description="질문하기")


def on_submit(btn):
    query = question_box.value
    if query.strip().lower() == "exit":
        with output_box:
            clear_output()
            print("종료합니다.")
        return
    with output_box:
        clear_output()
        print("답변을 생성하고 있습니다...")
        result = qa_chain({"query": query})
        print("\n[답변]:", result["result"])
        print("=" * 80)
        print("[참고 문서]")
        for i, doc in enumerate(result["source_documents"]):
            print(f"\n[{i+1}] {doc.page_content[:300]}...")


submit_btn.on_click(on_submit)
display(question_box, submit_btn, output_box)

Text(value='', description='질문:', placeholder='여기에 질문을 입력하세요')

Button(description='질문하기', style=ButtonStyle())

Output()

8-2 답변

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

question_box = widgets.Text(
    value="",
    placeholder="여기에 질문을 입력하세요",
    description="질문:",
    disabled=False,
)
output_box = widgets.Output()
submit_btn = widgets.Button(description="질문하기")


def on_submit(btn):
    query = question_box.value
    with output_box:
        clear_output()
        if not query.strip():
            print("질문을 입력해 주세요!")
            return
        if query.strip().lower() == "exit":
            print("종료합니다.")
            return
        print("답변을 생성하고 있습니다...")
        try:
            result = qa_chain({"query": query})
            print(
                "\n[답변]:", result.get("result", "응답이 없습니다.")[:1000]
            )  # 답변 1000자 제한 예시
            print("=" * 80)
            print("[참고 문서]")
            for i, doc in enumerate(
                result.get("source_documents", [])[:3]
            ):  # 3개까지만 미리보기
                print(f"\n[{i+1}] {doc.page_content[:500]}...")
        except Exception as e:
            print("답변 생성 중 오류 발생:", e)


submit_btn.on_click(on_submit)
display(question_box, submit_btn, output_box)

Text(value='', description='질문:', placeholder='여기에 질문을 입력하세요')

Button(description='질문하기', style=ButtonStyle())

Output()

# 현재 코드와 End-to-End 활용성

## 1. End-to-End 파이프라인이란?

- **질문 → 관련 문서 검색 → 답변 생성**
- 이 전체 과정을 한 번에 자동화한 것이 End-to-End 방식

---

## 2. 현재 코드 흐름

### 1) 데이터 전처리 및 임베딩

- 여러 소스에서 데이터를 수집하고 전처리(필터링) 후,
- `KoSBERT` 임베딩 → **FAISS 벡터DB에 저장**
    - 이 과정이 검색(검색 DB 구축) 역할을 함

### 2) 질문 입력 & QA 체인 연결

- 사용자가 `ipywidgets`로 질문을 입력하면,
- **RAG 파이프라인**이
    1. 질문을 임베딩하여 FAISS에서 **관련 문서(컨텍스트) 검색**
    2. 검색된 문서들을 LLM(예: Mistral-7B)에 넣어서 **최종 답변 생성**
- 사용자는 중간 과정 없이 곧바로 답변을 받음  
    → **이게 바로 End-to-End 흐름**

### 3) End-to-End 데이터셋 분리

- 별도로 준비한 End-to-End 학습용 데이터셋  
    (질문/문서/답변 페어)
- ColBERT 등 파인튜닝, 성능 평가, 추가 실험에도 바로 사용 가능

---

## 3. 활용 관점

- **실시간 질문-답변**  
    중간 수동 개입 없이, 사용자가 질문만 하면  
    “검색 → 답변 생성”까지 바로 진행

- **파인튜닝 데이터**  
    End-to-End 구조로 학습 데이터를 만들어  
    LLM 추가 개선, 평가 등도 쉽게 진행 가능

---

## 결론

- **End-to-End RAG 구조가 전체 파이프라인에서 자연스럽게 동작하고 있습니다.**
- 현재 코드는 RAG 구조(검색+생성)와 End-to-End 철학에 모두 부합합니다.
- 실제 서비스, 프로토타입, 실험, 파인튜닝, 응용 등 다양한 곳에서  
  **효과적으로 활용할 수 있습니다!**